<a href="https://colab.research.google.com/github/RohEunSeo/LGAimers/blob/main/test_data%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
ROOT_DIR = "/content/drive/MyDrive/lgAimers.5(updated.ver)"
RANDOM_STATE = 110

# 파일 경로 설정
test_data_path = os.path.join(ROOT_DIR, "test.csv")

# 데이터 읽어오기
test_data = pd.read_csv(test_data_path)

test_data

,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,0001be084fbc4aaa9d921f39e595961b,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1,OK,1000.0,NaN,...,195,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
1,0005bbd180064abd99e63f9ed3e1ac80,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1,OK,1000.0,NaN,...,14,NaN,NaN,256,NaN,NaN,1,NaN,NaN,NaN
2,000948934c4140d883d670adcb609584,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,1,OK,240.0,NaN,...,98,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
3,000a6bfd02874c6296dc7b2e9c5678a7,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1,OK,1000.0,NaN,...,14,NaN,NaN,0,NaN,NaN,1,NaN,NaN,NaN
4,0018e78ce91343678716e2ea27a51c95,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,1,OK,240.0,NaN,...,1,NaN,NaN,215,NaN,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1,OK,1000.0,NaN,...,14,NaN,NaN,131,NaN,NaN,1,NaN,NaN,NaN
17357,ffed8923c8a448a98afc641b770be153,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1,OK,1000.0,NaN,...,12,NaN,NaN,279,NaN,NaN,1,NaN,NaN,NaN
17358,fff1e73734da40adbe805359b3efb462,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,1,OK,240.0,NaN,...,4,NaN,NaN,66,NaN,NaN,1,NaN,NaN,NaN
17359,fff8e38bdd09470baf95f71e92075dec,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,1,OK,240.0,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN


# 결측치 제거(모든 행이 Nan값인 열 제거, target 제외)

In [ ]:
# 결측치 확인
missing_values = test_data.isnull().sum()

# 결측치가 있는 열들만 출력
missing_columns = missing_values[missing_values > 0]
print(f"Columns with missing values:\n{missing_columns}")

# 모든 행이 결측치인 열들 중에서 target 열을 제외
columns_to_drop = missing_values[(missing_values == len(test_data)) & (missing_values.index != 'target')].index

# 지정된 열을 삭제
test_data_cleaned = test_data.drop(columns=columns_to_drop)

Columns with missing values:
CURE END POSITION X Unit Time_Dam      17361
CURE END POSITION X Judge Value_Dam    17361
CURE END POSITION Z Unit Time_Dam      17361
CURE END POSITION Z Judge Value_Dam    17361
CURE END POSITION Θ Unit Time_Dam      17361
                                       ...  
Receip No Unit Time_Fill2              17361
Receip No Judge Value_Fill2            17361
WorkMode Unit Time_Fill2               17361
WorkMode Judge Value_Fill2             17361
target                                 17361
Length: 287, dtype: int64


In [ ]:
test_data_cleaned

,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,0001be084fbc4aaa9d921f39e595961b,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1,OK,1000.0,12.5,...,91.8,270.0,50,85,19.8,13.0,195,1,0,NaN
1,0005bbd180064abd99e63f9ed3e1ac80,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1,OK,1000.0,12.5,...,50.0,91.8,270,50,85.0,19.8,14,256,1,NaN
2,000948934c4140d883d670adcb609584,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,1,OK,240.0,2.5,...,91.8,270.0,50,85,19.7,1.0,98,1,0,NaN
3,000a6bfd02874c6296dc7b2e9c5678a7,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1,OK,1000.0,12.5,...,50.0,91.8,270,50,85.0,20.0,14,0,1,NaN
4,0018e78ce91343678716e2ea27a51c95,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,1,OK,240.0,2.5,...,50.0,91.8,270,50,85.0,19.8,1,215,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1,OK,1000.0,12.5,...,50.0,91.8,270,50,85.0,19.5,14,131,1,NaN
17357,ffed8923c8a448a98afc641b770be153,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1,OK,1000.0,12.5,...,50.0,91.8,270,50,85.0,19.8,12,279,1,NaN
17358,fff1e73734da40adbe805359b3efb462,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,1,OK,240.0,2.5,...,50.0,91.8,270,50,85.0,20.5,4,66,1,NaN
17359,fff8e38bdd09470baf95f71e92075dec,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,1,OK,240.0,2.5,...,91.8,270.0,50,85,18.9,1.0,117,1,0,NaN


# 결측치 제거(모든 열이 0인 경우)

In [ ]:
# 모든 값이 0인 열 식별
columns_to_drop = test_data_cleaned.columns[(test_data_cleaned == 0).all()]

# 해당 열들을 삭제
test_data_cleaned = test_data_cleaned.drop(columns=columns_to_drop)

# 삭제된 열들의 이름 확인
deleted_columns = columns_to_drop.tolist()
print(f"삭제된 열: {deleted_columns}")

삭제된 열: ['CURE STANDBY POSITION Θ Collect Result_Dam', 'CURE STANDBY POSITION Θ Collect Result_Fill2', 'DISCHARGED SPEED OF RESIN Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill2', 'Dispense Volume(Stage1) Collect Result_Fill2', 'Dispense Volume(Stage2) Collect Result_Fill2', 'Dispense Volume(Stage3) Collect Result_Fill2']


In [ ]:
test_data_cleaned

,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,0001be084fbc4aaa9d921f39e595961b,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1,OK,1000.0,12.5,...,91.8,270.0,50,85,19.8,13.0,195,1,0,NaN
1,0005bbd180064abd99e63f9ed3e1ac80,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1,OK,1000.0,12.5,...,50.0,91.8,270,50,85.0,19.8,14,256,1,NaN
2,000948934c4140d883d670adcb609584,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,1,OK,240.0,2.5,...,91.8,270.0,50,85,19.7,1.0,98,1,0,NaN
3,000a6bfd02874c6296dc7b2e9c5678a7,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1,OK,1000.0,12.5,...,50.0,91.8,270,50,85.0,20.0,14,0,1,NaN
4,0018e78ce91343678716e2ea27a51c95,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,1,OK,240.0,2.5,...,50.0,91.8,270,50,85.0,19.8,1,215,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1,OK,1000.0,12.5,...,50.0,91.8,270,50,85.0,19.5,14,131,1,NaN
17357,ffed8923c8a448a98afc641b770be153,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1,OK,1000.0,12.5,...,50.0,91.8,270,50,85.0,19.8,12,279,1,NaN
17358,fff1e73734da40adbe805359b3efb462,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,1,OK,240.0,2.5,...,50.0,91.8,270,50,85.0,20.5,4,66,1,NaN
17359,fff8e38bdd09470baf95f71e92075dec,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,1,OK,240.0,2.5,...,91.8,270.0,50,85,18.9,1.0,117,1,0,NaN


# 결측치 제거(nan값과 Ok만 있는 경우)

In [ ]:
# 삭제할 열 목록
columns_to_drop = [
    "Wip Line_Dam",
    "Wip Line_AutoClave",
    "Wip Line_Fill1",
    "Wip Line_Fill2",
    "Insp. Seq No._Dam",
    "Insp Judge Code_Dam",
    "Insp. Seq No._AutoClave",
    "Insp Judge Code_AutoClave",
    "Insp. Seq No._Fill1",
    "Insp Judge Code_Fill1",
    "Insp. Seq No._Fill2",
    "Insp Judge Code_Fill2",
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam',
    'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave',
    'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2',
    "1st Pressure Judge Value_AutoClave",
    "2nd Pressure Judge Value_AutoClave",
    "3rd Pressure Judge Value_AutoClave",
    "PalletID Collect Result_Dam",
    "Receip No Collect Result_Dam",
    "PalletID Collect Result_Fill1",
    "Receip No Collect Result_Fill1",
    "PalletID Collect Result_Fill2",
    "Receip No Collect Result_Fill2"


]

# 열 삭제
test_data_cleaned.drop(columns=columns_to_drop, inplace=True)

In [ ]:
test_data_cleaned

,Set ID,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION X Collect Result_Dam,...,Head Clean Position X Collect Result_Fill2,Head Clean Position Y Collect Result_Fill2,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,Production Qty Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,0001be084fbc4aaa9d921f39e595961b,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1000.0,12.5,90,70,1150,...,119,50,91.8,270.0,50,85,19.8,195,0,NaN
1,0005bbd180064abd99e63f9ed3e1ac80,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,14,1,NaN
2,000948934c4140d883d670adcb609584,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,240.0,2.5,-90,70,1150,...,119,50,91.8,270.0,50,85,19.7,98,0,NaN
3,000a6bfd02874c6296dc7b2e9c5678a7,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,14,1,NaN
4,0018e78ce91343678716e2ea27a51c95,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,240.0,2.5,-90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,14,1,NaN
17357,ffed8923c8a448a98afc641b770be153,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,12,1,NaN
17358,fff1e73734da40adbe805359b3efb462,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,240.0,2.5,-90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,4,1,NaN
17359,fff8e38bdd09470baf95f71e92075dec,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,240.0,2.5,-90,70,1150,...,119,50,91.8,270.0,50,85,18.9,117,0,NaN


In [ ]:
# 남은 결측치가 있는 열과 결측치 개수 확인
remaining_missing_values_cleaned = test_data_cleaned.isnull().sum()
remaining_missing_columns_cleaned = remaining_missing_values_cleaned[remaining_missing_values_cleaned > 0]

# 결측치가 있는 열의 개수 출력
print(f"\nNumber of columns with missing values: {len(remaining_missing_columns_cleaned)}")

# 결측치가 있는 열과 결측치 개수 출력
print("\nMissing values in each column:")
for column, missing_count in remaining_missing_columns_cleaned.items():
    print(f"{column}: {missing_count} missing values")


Number of columns with missing values: 4

Missing values in each column:
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam: 5468 missing values
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1: 5468 missing values
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2: 5468 missing values
target: 17361 missing values


In [ ]:
test_data_cleaned

,Set ID,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION X Collect Result_Dam,...,Head Clean Position X Collect Result_Fill2,Head Clean Position Y Collect Result_Fill2,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,Production Qty Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,0001be084fbc4aaa9d921f39e595961b,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1000.0,12.5,90,70,1150,...,119,50,91.8,270.0,50,85,19.8,195,0,NaN
1,0005bbd180064abd99e63f9ed3e1ac80,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,14,1,NaN
2,000948934c4140d883d670adcb609584,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,240.0,2.5,-90,70,1150,...,119,50,91.8,270.0,50,85,19.7,98,0,NaN
3,000a6bfd02874c6296dc7b2e9c5678a7,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,14,1,NaN
4,0018e78ce91343678716e2ea27a51c95,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,240.0,2.5,-90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,14,1,NaN
17357,ffed8923c8a448a98afc641b770be153,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,12,1,NaN
17358,fff1e73734da40adbe805359b3efb462,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,240.0,2.5,-90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,4,1,NaN
17359,fff8e38bdd09470baf95f71e92075dec,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,240.0,2.5,-90,70,1150,...,119,50,91.8,270.0,50,85,18.9,117,0,NaN


# 결측치 대체

In [ ]:
# OK 값을 NaN으로 변경할 열 목록
columns_to_modify = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
]

# OK 값을 NaN으로 변경
for column in columns_to_modify:
    test_data_cleaned[column] = test_data_cleaned[column].replace('OK', pd.NA)
# 모든 결측치를 NaN으로 통일
train_data_cleaned = test_data_cleaned.replace({pd.NA: np.nan})

# 모든 열을 숫자로 변환하고, 변환할 수 없는 값을 NaN으로 처리
columns_to_convert = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1'
]

for column in columns_to_convert:
    test_data_cleaned[column] = pd.to_numeric(test_data_cleaned[column], errors='coerce')

In [ ]:
test_data_cleaned

,Set ID,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION X Collect Result_Dam,...,Head Clean Position X Collect Result_Fill2,Head Clean Position Y Collect Result_Fill2,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,Production Qty Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,0001be084fbc4aaa9d921f39e595961b,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1000.0,12.5,90,70,1150,...,119,50,91.8,270.0,50,85,19.8,195,0,NaN
1,0005bbd180064abd99e63f9ed3e1ac80,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,14,1,NaN
2,000948934c4140d883d670adcb609584,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,240.0,2.5,-90,70,1150,...,119,50,91.8,270.0,50,85,19.7,98,0,NaN
3,000a6bfd02874c6296dc7b2e9c5678a7,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,14,1,NaN
4,0018e78ce91343678716e2ea27a51c95,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,240.0,2.5,-90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,14,1,NaN
17357,ffed8923c8a448a98afc641b770be153,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,12,1,NaN
17358,fff1e73734da40adbe805359b3efb462,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,240.0,2.5,-90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,4,1,NaN
17359,fff8e38bdd09470baf95f71e92075dec,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,240.0,2.5,-90,70,1150,...,119,50,91.8,270.0,50,85,18.9,117,0,NaN


# 결측치 대체(Dam, Fill1, Fill2)

In [ ]:
# Stage1 Collect Result Fill2에서 NaN 값을 대체하기 위한 조건 정의
condition_1 = (test_data_cleaned['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2'] == 304.8) & (test_data_cleaned['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'] == 499.8)
condition_2 = (test_data_cleaned['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2'] == 305.0) & (test_data_cleaned['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'] == 499.8)
condition_3 = (test_data_cleaned['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2'] == 835.5) & (test_data_cleaned['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'] == 458)

# 정의된 조건에 따라 Stage1의 NaN 값을 각각 대체
test_data_cleaned.loc[condition_1 & test_data_cleaned['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'].isna(), 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] = 694.2
test_data_cleaned.loc[condition_2 & test_data_cleaned['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'].isna(), 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] = 694
test_data_cleaned.loc[condition_3 & test_data_cleaned['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'].isna(), 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] = 156

# 결측치 대체 함수: Dam_Stage1
def fill_within_range_dam(test_data_cleaned, target_column, stage2_column, stage3_column, valid_values, valid_range):
    for index in test_data_cleaned[target_column].index:
        if pd.isna(test_data_cleaned.at[index, target_column]):
            for value in valid_values:
                temp_sum = (value +
                            test_data_cleaned.at[index, stage2_column] +
                            test_data_cleaned.at[index, stage3_column])
                if valid_range[0] <= temp_sum <= valid_range[1]:
                    test_data_cleaned.at[index, target_column] = value
                    break
    return test_data_cleaned

# Dam의 Stage1 결측치 대체 (가능한 값들만 사용)
valid_dam_values = [550.3, 550, 549.5, 549, 548.5, 162.4]  # Dam_Stage1에 존재하는 값들
test_data_cleaned = fill_within_range_dam(
    test_data_cleaned,
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
    valid_dam_values,
    (1170.7, 1180.5)
)
# Fill1의 결측치 대체에서 Stage2, Stage3를 변경하지 않고, 중복되는 경우 동일한 값을 사용
def fill_within_range_fill1(test_data_cleaned, target_column, stage2_column, stage3_column, valid_stage1_values, valid_range):
    # 가능한 조합을 미리 계산하여 저장
    fill1_replacement_dict = {}

    for index in test_data_cleaned[target_column].index:
        if pd.isna(test_data_cleaned.at[index, target_column]):
            stage2_value = test_data_cleaned.at[index, stage2_column]
            stage3_value = test_data_cleaned.at[index, stage3_column]
            key = (stage2_value, stage3_value)

            if key not in fill1_replacement_dict:
                possible_values = []
                for value1 in valid_stage1_values:
                    temp_sum = value1 + stage2_value + stage3_value
                    if valid_range[0] <= temp_sum <= valid_range[1]:
                        possible_values.append(value1)

                if possible_values:
                    # 첫 번째 가능한 값을 선택 (다른 정책을 사용할 수도 있음)
                    chosen_value = possible_values[0]
                    fill1_replacement_dict[key] = chosen_value

            if key in fill1_replacement_dict:
                test_data_cleaned.at[index, target_column] = fill1_replacement_dict[key]

    return test_data_cleaned
# 사용 가능한 Stage1의 값들 (제한된 범위)
valid_fill1_values1 = [838.4, 837.9, 837.7, 838.2, 837.5, 157, 158, 156.1]

# Fill1의 Stage1 결측치 대체
test_data_cleaned = fill_within_range_fill1(
    test_data_cleaned,
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1',
    valid_fill1_values1,
    (1453, 1457)
)
test_data_cleaned.dropna(subset=['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'], inplace=True)

missing_fill1 = test_data_cleaned['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'].isna().sum()
print(f"Fill1의 결측치 대체 후 남은 결측치 개수: {missing_fill1}")


Fill1의 결측치 대체 후 남은 결측치 개수: 0


In [ ]:
test_data_cleaned

,Set ID,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION X Collect Result_Dam,...,Head Clean Position X Collect Result_Fill2,Head Clean Position Y Collect Result_Fill2,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,Production Qty Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,0001be084fbc4aaa9d921f39e595961b,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1000.0,12.5,90,70,1150,...,119,50,91.8,270.0,50,85,19.8,195,0,NaN
1,0005bbd180064abd99e63f9ed3e1ac80,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,14,1,NaN
2,000948934c4140d883d670adcb609584,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,240.0,2.5,-90,70,1150,...,119,50,91.8,270.0,50,85,19.7,98,0,NaN
3,000a6bfd02874c6296dc7b2e9c5678a7,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,14,1,NaN
4,0018e78ce91343678716e2ea27a51c95,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,240.0,2.5,-90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,14,1,NaN
17357,ffed8923c8a448a98afc641b770be153,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1000.0,12.5,90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,12,1,NaN
17358,fff1e73734da40adbe805359b3efb462,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,240.0,2.5,-90,70,1150,...,-10,119,50.0,91.8,270,50,85.0,4,1,NaN
17359,fff8e38bdd09470baf95f71e92075dec,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,240.0,2.5,-90,70,1150,...,119,50,91.8,270.0,50,85,18.9,117,0,NaN


# Champer 온도 데이터 파생변수 생성

In [ ]:
# 온도가 50도 이상인지 여부를 나타내는 파생변수 생성
'''
test_data_cleaned['Is_Temp_Above_50'] = test_data_cleaned['Chamber Temp. Collect Result_AutoClave'].apply(lambda x: 1 if x >= 50 else 0)

# OK일 때 온도 차이 변수 생성 (50도 이상이어야 OK이므로 이 차이를 계산)
test_data_cleaned['Temp_Diff_OK'] = test_data_cleaned.apply(
    lambda row: row['Chamber Temp. Collect Result_AutoClave'] - 50 if row['Chamber Temp. Judge Value_AutoClave'] == 'OK' else 0, axis=1
)

# NG일 때 온도 차이 변수 생성 (49도 이하이어야 NG이므로 이 차이를 계산)
test_data_cleaned['Temp_Diff_NG'] = test_data_cleaned.apply(
    lambda row: 50 - row['Chamber Temp. Collect Result_AutoClave'] if row['Chamber Temp. Judge Value_AutoClave'] == 'NG' else 0, axis=1
)'''


In [ ]:
test_data_cleaned

,Set ID,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION X Collect Result_Dam,...,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,Production Qty Collect Result_Fill2,WorkMode Collect Result_Fill2,target,Is_Temp_Above_50,Temp_Diff_OK,Temp_Diff_NG
0,0001be084fbc4aaa9d921f39e595961b,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1000.0,12.5,90,70,1150,...,270.0,50,85,19.8,195,0,NaN,1,4,0
1,0005bbd180064abd99e63f9ed3e1ac80,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1000.0,12.5,90,70,1150,...,91.8,270,50,85.0,14,1,NaN,1,6,0
2,000948934c4140d883d670adcb609584,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,240.0,2.5,-90,70,1150,...,270.0,50,85,19.7,98,0,NaN,1,4,0
3,000a6bfd02874c6296dc7b2e9c5678a7,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1000.0,12.5,90,70,1150,...,91.8,270,50,85.0,14,1,NaN,1,2,0
4,0018e78ce91343678716e2ea27a51c95,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,240.0,2.5,-90,70,1150,...,91.8,270,50,85.0,1,1,NaN,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1000.0,12.5,90,70,1150,...,91.8,270,50,85.0,14,1,NaN,1,0,0
17357,ffed8923c8a448a98afc641b770be153,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1000.0,12.5,90,70,1150,...,91.8,270,50,85.0,12,1,NaN,1,3,0
17358,fff1e73734da40adbe805359b3efb462,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,240.0,2.5,-90,70,1150,...,91.8,270,50,85.0,4,1,NaN,1,4,0
17359,fff8e38bdd09470baf95f71e92075dec,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,240.0,2.5,-90,70,1150,...,270.0,50,85,18.9,117,0,NaN,1,4,0


# 이상치 대체

In [ ]:
# IQR을 이용한 이상치 탐지 및 상한선/하한선으로 대체하는 함수
'''def cap_outliers_iqr(df, column, iqr_multiplier=3):
    # 1분위수(25%)와 3분위수(75%) 계산
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # IQR을 사용해 상한선과 하한선 계산 (3 * IQR로 설정)
    lower_bound = Q1 - iqr_multiplier * IQR
    upper_bound = Q3 + iqr_multiplier * IQR

    # 하한선보다 작은 값은 하한선으로, 상한선보다 큰 값은 상한선으로 대체
    df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
    df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])

# 수치형 데이터 선택
numeric_columns = test_data_cleaned.select_dtypes(include=[float, int]).columns

# 각 수치형 컬럼에 대해 IQR 방법으로 이상치를 상한선/하한선으로 대체
for column in numeric_columns:
    cap_outliers_iqr(test_data_cleaned, column, iqr_multiplier=3)'''

'def cap_outliers_iqr(df, column, iqr_multiplier=3):\n    # 1분위수(25%)와 3분위수(75%) 계산\n    Q1 = df[column].quantile(0.25)\n    Q3 = df[column].quantile(0.75)\n    IQR = Q3 - Q1\n\n    # IQR을 사용해 상한선과 하한선 계산 (3 * IQR로 설정)\n    lower_bound = Q1 - iqr_multiplier * IQR\n    upper_bound = Q3 + iqr_multiplier * IQR\n\n    # 하한선보다 작은 값은 하한선으로, 상한선보다 큰 값은 상한선으로 대체\n    df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])\n    df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])\n\n# 수치형 데이터 선택\nnumeric_columns = test_data_cleaned.select_dtypes(include=[float, int]).columns\n\n# 각 수치형 컬럼에 대해 IQR 방법으로 이상치를 상한선/하한선으로 대체\nfor column in numeric_columns:\n    cap_outliers_iqr(test_data_cleaned, column, iqr_multiplier=3)'

In [ ]:
# 이상치가 남아있는지 확인하는 코드
'''def detect_remaining_outliers(df, column, iqr_multiplier=3):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - iqr_multiplier * IQR
    upper_bound = Q3 + iqr_multiplier * IQR

    # 남아있는 이상치 탐지
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return len(outliers)

# 남아있는 이상치 확인
total_remaining_outliers = 0
for column in numeric_columns:
    remaining_outliers = detect_remaining_outliers(test_data_cleaned, column, iqr_multiplier=3)
    if remaining_outliers > 0:
        print(f"'{column}' 열에 남아있는 이상치 개수: {remaining_outliers}개")
        total_remaining_outliers += remaining_outliers

if total_remaining_outliers == 0:
    print("이상치가 더 이상 존재하지 않습니다.")
else:
    print(f"총 {total_remaining_outliers}개의 이상치가 남아 있습니다.")'''

'def detect_remaining_outliers(df, column, iqr_multiplier=3):\n    Q1 = df[column].quantile(0.25)\n    Q3 = df[column].quantile(0.75)\n    IQR = Q3 - Q1\n\n    lower_bound = Q1 - iqr_multiplier * IQR\n    upper_bound = Q3 + iqr_multiplier * IQR\n\n    # 남아있는 이상치 탐지\n    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]\n    return len(outliers)\n\n# 남아있는 이상치 확인\ntotal_remaining_outliers = 0\nfor column in numeric_columns:\n    remaining_outliers = detect_remaining_outliers(test_data_cleaned, column, iqr_multiplier=3)\n    if remaining_outliers > 0:\n        print(f"\'{column}\' 열에 남아있는 이상치 개수: {remaining_outliers}개")\n        total_remaining_outliers += remaining_outliers\n\nif total_remaining_outliers == 0:\n    print("이상치가 더 이상 존재하지 않습니다.")\nelse:\n    print(f"총 {total_remaining_outliers}개의 이상치가 남아 있습니다.")'

In [ ]:
test_data_cleaned

,Set ID,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION X Collect Result_Dam,...,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,Production Qty Collect Result_Fill2,WorkMode Collect Result_Fill2,target,Is_Temp_Above_50,Temp_Diff_OK,Temp_Diff_NG
0,0001be084fbc4aaa9d921f39e595961b,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1000.0,12.5,90,70,1150,...,270.0,50,85,19.8,195,0,NaN,1,4,0
1,0005bbd180064abd99e63f9ed3e1ac80,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1000.0,12.5,90,70,1150,...,91.8,270,50,85.0,14,1,NaN,1,6,0
2,000948934c4140d883d670adcb609584,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,240.0,2.5,-90,70,1150,...,270.0,50,85,19.7,98,0,NaN,1,4,0
3,000a6bfd02874c6296dc7b2e9c5678a7,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1000.0,12.5,90,70,1150,...,91.8,270,50,85.0,14,1,NaN,1,2,0
4,0018e78ce91343678716e2ea27a51c95,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,240.0,2.5,-90,70,1150,...,91.8,270,50,85.0,1,1,NaN,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1000.0,12.5,90,70,1150,...,91.8,270,50,85.0,14,1,NaN,1,0,0
17357,ffed8923c8a448a98afc641b770be153,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1000.0,12.5,90,70,1150,...,91.8,270,50,85.0,12,1,NaN,1,3,0
17358,fff1e73734da40adbe805359b3efb462,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,240.0,2.5,-90,70,1150,...,91.8,270,50,85.0,4,1,NaN,1,4,0
17359,fff8e38bdd09470baf95f71e92075dec,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,240.0,2.5,-90,70,1150,...,270.0,50,85,18.9,117,0,NaN,1,4,0


# 데이터 스케일링 Min-Max

In [ ]:
# Min-Max 스케일러 초기화
scaler = MinMaxScaler()

# 수치형 컬럼 선택
numeric_cols = test_data_cleaned.select_dtypes(exclude=['object']).columns

# train_data_cleaned에 Min-Max 스케일링 적용
test_data_cleaned[numeric_cols] = scaler.fit_transform(test_data_cleaned[numeric_cols])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:481: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [ ]:
test_data_cleaned

,Set ID,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION X Collect Result_Dam,...,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,Production Qty Collect Result_Fill2,WorkMode Collect Result_Fill2,target,Is_Temp_Above_50,Temp_Diff_OK,Temp_Diff_NG
0,0001be084fbc4aaa9d921f39e595961b,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.021669,0.401235,0.000000,NaN,1.0,0.814815,0.0
1,0005bbd180064abd99e63f9ed3e1ac80,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.694444,0.028807,0.058824,NaN,1.0,0.888889,0.0
2,000948934c4140d883d670adcb609584,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.020637,0.201646,0.000000,NaN,1.0,0.814815,0.0
3,000a6bfd02874c6296dc7b2e9c5678a7,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.694444,0.028807,0.058824,NaN,1.0,0.740741,0.0
4,0018e78ce91343678716e2ea27a51c95,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.694444,0.002058,0.058824,NaN,1.0,0.814815,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.694444,0.028807,0.058824,NaN,1.0,0.666667,0.0
17357,ffed8923c8a448a98afc641b770be153,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.694444,0.024691,0.058824,NaN,1.0,0.777778,0.0
17358,fff1e73734da40adbe805359b3efb462,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.694444,0.008230,0.058824,NaN,1.0,0.814815,0.0
17359,fff8e38bdd09470baf95f71e92075dec,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.012382,0.240741,0.000000,NaN,1.0,0.814815,0.0


In [ ]:
test_data_cleaned.to_csv("/content/drive/MyDrive/lgAimers.5(updated.ver)/modeling/test_data_cleaned(파생변수추가).csv", index = False)